In [1]:
import numpy as np
from scipy.spatial.distance import pdist
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy import integrate

## Distribution types

In [2]:
#Sampples 2 Normal Distributions.
def sample_normal(mu1, sigma1, mu2, sigma2, n):
    np.random.seed(0)
    x1 = np.random.normal(mu1, sigma1, n)
    x2 = np.random.normal(mu2, sigma2, n)
    return x1, x2

#Samples 2 Uniform Distributions.
def sample_uniform(low1, high1, low2, high2, n):
    np.random.seed(0)
    x1 = np.random.uniform(low1, high1, n)
    x2 = np.random.uniform(low2, high2, n)
    return x1, x2

#The pareto function is the inverse of the power function i.e. this is to test when two tail functions meet.
def sample_pareto_power(a, n):
    np.random.seed(0)
    x1 = np.random.pareto(a, n)
    x2 = np.random.power(a, n)
    return x1, x2

## Metrics

In [3]:
# Intersection over Union defined as: Area of Overlap / Area of Union
# The closer to 1 the better. 
def IOU(estimated_interval, true_interval):
    intersection = max(0, min(estimated_interval[1], true_interval[1]) - max(estimated_interval[0], true_interval[0]))
    union = max(estimated_interval[1], true_interval[1]) - min(estimated_interval[0], true_interval[0])
    
    if union == 0:
        return 1
    
    iou = intersection / union
    return iou

#https://www.researchgate.net/publication/350834499_Common_Limitations_of_Image_Processing_Metrics_A_Picture_Story
# Dice similarity coefficient (DSC) = F1 score: a harmonic mean of precision and recall. In other words, 
# it is calculated by 2 * intersection divided by the total number of pixel in both images.
# Closer to 1 the better 
def DSC(estimated_interval, true_interval):
    intersect = max(0, min(estimated_interval[1], true_interval[1]) - max(estimated_interval[0], true_interval[0]))
    total_sum = max(0, (estimated_interval[1] - estimated_interval[0])) +  max(0, (true_interval[1] - true_interval[0]))
    if total_sum == 0:
        return 1
    dice = np.mean(2*intersect/total_sum)
    return round(dice, 3) #round up to 3 decimal places

In [4]:
from sklearn.metrics import confusion_matrix
def iou_acc_multiple_dim(X, overlap, y_true):
    y_pred = []

    for data in X:
        if np.any(np.isclose(data, overlap)):
            y_pred.append(1)
        else:
            y_pred.append(0)

    CM = confusion_matrix(y_true, y_pred)
    

    if len(CM[0]) == 1:
        return 1, 1, y_pred

    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    
    #Calculate Score
    iou = TP/(TP+FP+FN)
    #print("Score: IOU", iou)
    acc= (TP+TN)/(TN + FN + TP + FP)
    #print("Score: Acc", acc)
    return iou, acc, y_pred

## True Overlap Evaluation

In [5]:
# Bin the data for full_x1 and full_x2 and get the overlap region where they are both greater than epsilon 
def get_overlap(full_x1, full_x2, epsilon):
    bins = np.linspace(np.min((np.min(full_x1), np.min(full_x2))), np.max((np.max(full_x1), np.max(full_x2))), 200)
    x1 = np.histogram(full_x1, bins=bins, density=True)[0]
    x2 = np.histogram(full_x2, bins=bins, density=True)[0]
    overlap = (x1 > epsilon) & (x2 > epsilon)
    return bins[1:][overlap]

In [6]:
def true_overlap(x1, x2, epsilon):
    #Get true overlap
    overlap = get_overlap(x1, x2, epsilon)
    true_interval = (0,0)
    if len(overlap) == 0: #If no true overlap
        true_interval == (0, 0)
    else: #If overlap
        true_interval = (overlap.min(), overlap.max())
    return true_interval
        
def estimated_overlap(overlap_nn):

    if len(overlap_nn) == 0: #If no estimated overlap
        estimated_interval = (0, 0)
    else: #If estimated overlap
        estimated_interval = (overlap_nn.min(), overlap_nn.max())
    return estimated_interval

## Evaluate Distance between Points

In [7]:
def compute_average_distance(X):
    """
    Computes the average distance among a set of n points in the d-dimensional space.

    Arguments:
        X {numpy array} - the query points in an array of shape (n,d), 
                          where n is the number of points and d is the dimension.
    Returns:
        {float} - the average distance among the points
    """
    distances = pdist(X)
    return np.mean(distances), np.min(distances), np.max(distances)

## Generation of the Data

In [8]:
def generate_data(n):
    distribution_types = [sample_normal, sample_uniform, sample_pareto_power]

    rand_distribution_index = np.random.randint(len(distribution_types))
    rng = np.random.default_rng()
    
    if rand_distribution_index == 0:
        mu1, sigma1, mu2, sigma2 = 3*rng.random((4, 1)) 
        x1, x2 = distribution_types[rand_distribution_index](mu1, sigma1, mu2, sigma2, n)
    if rand_distribution_index == 1:
        low1, high1, low2, high2 = 5*rng.random((4, 1))
        
        if low1 > high1:
            low1, high1 = high1, low1
        if low2 > high2:
            low2, high2 = high2, low2
            
        x1, x2 = distribution_types[rand_distribution_index](low1, high1, low2, high2, n)
    if rand_distribution_index == 2:
        a = 10*rng.random() 
        x1, x2 = distribution_types[rand_distribution_index](a, n)
        
    return x1, x2

## Graphs

In [9]:
def plot_x1_x2(x1, x2):
    plt.hist(x1, bins=30, alpha=0.5, density=True,label='x1')
    plt.hist(x2, bins=30, alpha=0.5, density=True, label='x2')
    plt.legend()

In [10]:
def graph(x1, x2, params,estimated_interval, true_interval, epsilon):
    fig, ax = plt.subplots()
    f1_distribution = stats.norm.pdf(x=x1, loc=params[0], scale=params[1])
    f2_distribution = stats.norm.pdf(x=x2, loc=params[2], scale=params[3])
    # Plot the distribution of two classes, overlap region and overlap points
    plt.scatter(x1, f1_distribution, alpha=0.3)
    plt.scatter(x2, f2_distribution, alpha=0.3)

    ax.axvspan(estimated_interval[0], estimated_interval[1], alpha=0.3, color='red')
    print('{} - Overlap region: [{}, {}]'.format("nn", estimated_interval[0], estimated_interval[1]))

    ax.set_xlim(np.min(np.concatenate([x1, x2])), np.max(np.concatenate([x1, x2])))

    # Plot true intervala
    ax.axvspan(true_interval[0], true_interval[1], alpha=0.3, color='blue')

    # Compute Intersection over Union between estimated interval and true interval
    iou = IOU(estimated_interval, true_interval)
    ax.set_title("Estimated overlap compared to True overlap")
    plt.xlabel("Value")
    plt.ylabel("Count")
    plt.show()

## Multi-Dimensions - Multi-Varialbles

In [11]:
def get_overlap_matrix(full_x1, full_x2, epsilon: float, bin_size: int) -> (np.ndarray, np.ndarray):
    """
    Find the true overlap of two given classes by check the distributions of the data
    Return two matrix, overlap matrix stores overlap region,
    bins matrix store coordinates of each element in overlap
    """
    # Reshape the features if it's one dimension
    if len(full_x1.shape) == 1:
        full_x1 = full_x1.reshape((full_x1.shape[0], 1))
    if len(full_x2.shape) == 1:
        full_x2 = full_x2.reshape((full_x2.shape[0], 1))
    # Get the max and min on each dimension
    x1_min = [np.min(full_x1[:, i]) for i in range(full_x1.shape[1])]
    x2_min = [np.min(full_x2[:, i]) for i in range(full_x2.shape[1])]
    x1_max = [np.max(full_x1[:, i]) for i in range(full_x1.shape[1])]
    x2_max = [np.max(full_x2[:, i]) for i in range(full_x2.shape[1])]
    # Get the min and max in both classes for the bounds of bins
    bins_min = [np.minimum(x1_min[i], x2_min[i]) for i in range(len(x1_min))]
    bins_max = [np.maximum(x1_max[i], x2_max[i]) for i in range(len(x1_max))]
    bin_ranges = []
    # Get the lower and upper bound for each dimension
    for i in range(len(x1_min)):
        bin_ranges.append((bins_min[i], bins_max[i]))
    # Histogram process
    x1 = np.histogramdd(full_x1, bins=bin_size - 1, density=True, range=bin_ranges)[0]
    x2 = np.histogramdd(full_x2, bins=bin_size - 1, density=True, range=bin_ranges)[0]
    # Get overlap matrix
    overlap = (x1 > epsilon) & (x2 > epsilon)
    # Get coordinates index
    bins = np.linspace(bins_min, bins_max, bin_size)
    return overlap, bins

def get_overlapped_data(x1, x2, bins, overlap) -> np.ndarray:
    """
    Filter the given datasets and return the data in the overlap region
    """
    overlapped_data = []
    for x in np.concatenate((x1, x2)):
        coordinates = get_coordinates(x, bins)
        overlapped = overlap
        # Get the boolean value in overlap matrix
        for coordinate in coordinates:
            overlapped = overlapped[coordinate]
        if overlapped:
            # Store the overlapped points
            overlapped_data.append(x.tolist())
    if len(overlapped_data) == 0:
        return np.reshape(np.arange(0), (0, len(x1[0])))
    else:
        return np.reshape(overlapped_data, (len(overlapped_data), len(overlapped_data[0])))

def get_coordinates(data, bins) -> list:
    """
    Find the coordinates of given data in the overlap matrix
    """
    res = []
    # Loop for each dimension
    for i in range(len(data)):
        value = data[i]
        index = get_index(value, 0, len(bins) - 1, i, bins)
        res.append(index)
    return res

def get_index(value, l, r, dim_index, bins) -> int:
    """
    Using binary search to find the index of given value
    """
    if r - l == 1:
        return l
    mid = round((l + r) / 2)
    mid_value = bins[mid][dim_index]
    if mid_value > value:
        return get_index(value, l, mid, dim_index, bins)
    else:
        return get_index(value, mid, r, dim_index, bins)
    

def graph_n_dim(X, y_true, y_pred, epsilon, title):
        
    n = len(X[0])
    if n == 2:
        sns.set(rc={'figure.figsize':(14,10)})
        sns.kdeplot(x=X[:, 0], y= X[:, 1], cmap="Reds", shade=True, thresh=epsilon, alpha = 1)
        sns.scatterplot(x=X[:, 0], y= X[:, 1], style=y_true==y_pred, alpha = 0.5)
        plt.title(title, fontsize = 30)
        plt.xlabel('Coordinates X1', fontsize = 20) 
        plt.ylabel('Coordinates X2', fontsize = 20)
        plt.show()
    else:

        for i in range(n):
            plt.scatter(X[:, i], y_pred == y_true, c=y_true)
            plt.colorbar()
            plt.show()

In [13]:
def overlapping_points_using_stats_norm():
    eps = 0.1
    size = 1000
    mean1 = 0  
    mean2 = 2 
    scale1 = 1
    scale2 = 1

    x1 = np.random.normal(mean1, scale1, size)
    x2 = np.random.normal(mean2, scale2, size)
    X = np.concatenate([x1, x2]).reshape(-1, 1)

    # Get true distribution and overlap
    f1_distribution = stats.norm.pdf(x=x1, loc=mean1, scale=scale1)
    f2_distribution = stats.norm.pdf(x=x2, loc=mean2, scale=scale2)
    # Plot the distribution of two classes, overlap region and overlap points
    plt.scatter(x1, f1_distribution, alpha=0.3)
    plt.scatter(x2, f2_distribution, alpha=0.3)

    f1_distribution = stats.norm.pdf(x=X, loc=mean1, scale=scale1)
    f2_distribution = stats.norm.pdf(x=X, loc=mean2, scale=scale2)
    class_ov = (f1_distribution > eps) & (f2_distribution > eps)
    X_coor = X[class_ov]
    plt.scatter(X_coor, np.minimum(f1_distribution[class_ov], f2_distribution[class_ov]), alpha=1, c="red")
    plt.axhline(eps, color='red', linestyle="--", label="epsilon") # horizontal
    plt.legend(["x1", "x2", "overlapping points", 'epsilon'], fontsize=8)

    plt.xlabel("Coordinates")
    plt.ylabel("Density")
    plt.title("Overlapping points of two normaly distributed function")
    plt.show()

In [14]:
def overlapping_n_dim():
    eps = 0.03
    size = (500, 2)
    min = -4
    max = 4
    bin_size = 50
    step = 1 / bin_size
    mean1 = [-1, -1]  # Mean for x1 with 2 dimensions
    mean2 = [1, 1]  # Mean for x2 with 2 dimensions

    # Generate test data
    x1 = np.random.normal(-1, 1, size)
    x2 = np.random.normal(1, 1, size)
    X = np.concatenate([x1, x2]).reshape(-1, 2)
    # Distribution for classes
    f1 = stats.multivariate_normal.pdf(x=X, mean=mean1, allow_singular=True)
    f2 = stats.multivariate_normal.pdf(x=X, mean=mean2, allow_singular=True)
    # Get overlap points
    overlap = (f1 > eps) & (f2 > eps)
    ov_data = X[overlap]

    # Get true distribution and overlap
    xs, ys = np.mgrid[min:max:step, min:max:step]
    ts = np.dstack((xs, ys))
    f1_distribution = stats.multivariate_normal.pdf(x=ts, mean=mean1, allow_singular=True)
    f2_distribution = stats.multivariate_normal.pdf(x=ts, mean=mean2, allow_singular=True)
    class_ov = (f1_distribution > eps) & (f2_distribution > eps)

    # Plot the distribution of two classes, overlap region and overlap points
    plt.figure()
    plt.contour(xs, ys, f1_distribution, cmap='Reds')
    plt.contour(xs, ys, f2_distribution, cmap='Blues')
    plt.imshow(class_ov.T, extent=[min, max, min, max], origin='lower', cmap='plasma', alpha=0.8)
    plt.scatter(ov_data[:, 0], ov_data[:, 1], color="green", alpha=0.5)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.show()

In [15]:
# n: nr. of samples
# m: nr. of dimenions
def sample_normal_nd(mean1, mean2, sd1, sd2, n, m):
    x1 = np.random.normal(mean1, sd1, (n, m))
    x2 = np.random.normal(mean2, sd2, (n, m))

    return x1, x2

# n: nr. of samples
# m: nr. of dimenions
# outliers will be taking up a % of total samples
# outlier_chance: % of samples consisting of outliers (rest % is normal samples)
# outliermean: mean where outliers are (e.g. =5, outliers will be on (5,5) if in 2D)
# returns x1, x2 and non_outlier_matrix (a vector indicating which samples are not outliers with 1 and outliers with 0)
def sample_normal_outliers(mean1, mean2, sd1, sd2, n, m, outlier_chance, outliermean):
    true_samples1, true_samples2 = sample_normal_nd(mean1, mean2, sd1, sd2, n, m)
    outliers1, outliers2 = sample_normal_nd(outliermean, outliermean, sd1, sd2, n, m)
    bool_array = np.random.choice([0, 1], p=[outlier_chance, 1-outlier_chance], size=(n,1))
    bool_matrix = np.repeat(bool_array, m, axis=1)
    x1 = true_samples1 * bool_matrix + outliers1 * (1 - bool_matrix)
    x2 = true_samples2 * bool_matrix + outliers2 * (1 - bool_matrix)
    # binary matrix which indicates which samples are non-outliers (with 1 being non-outlier)
    non_outlier_matrix = np.concatenate([bool_matrix, bool_matrix]).reshape(-1, m)
    return x1, x2, non_outlier_matrix

# computing iou only if nonoutlier
def outlier_iou(prediction, y, non_outlier_matrix):
    tp = fp = fn = 0.0
    for p, t, n in zip(prediction, y, non_outlier_matrix):
        # if not an outlier
        if n == 1:
            if p == 1:
                if t == 1:
                    tp += 1
                else:
                    fp += 1
            else:
                if t == 1:
                    fn += 1
    if (tp + fp + fn) == 0:
        return 0
    return tp / (tp + fp + fn)